# 通し課題模範解答 回帰編 DAY 1
- kaggle の kickstarter project に関して，usd_pledged_real を予測するモデルを作成する
    - https://www.kaggle.com/kemical/kickstarter-projects?select=ks-projects-201801.csv
- DAY 1 では，以下を行う
    - データの読み込み
    - データの基礎集計
    - 有効な説明変数の選択
    - データの可視化及び欠損値・異常値の処理
    - 質的変数の処理
    - 線形回帰の利用
    - 予測精度の算出（検証については DAY 2）

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

## データの読み込み

In [2]:
df = pd.read_csv('../data/ks-projects-201801.csv', index_col='ID')

- 各列の概略は以下の通りである．
    - ID: 各プロジェクトに固有の ID である．誤って説明変数に利用しないように，`index_col='ID'`として行名に固定してしまうのが良い（後述）．
    - name: プロジェクトの名称
    - category: プロジェクトの小分類
    - main_category: プロジェクトの大分類
    - currency: プロジェクトで集められた支援金の通貨単位
    - deadline: プロジェクトの終了期日
    - goal: プロジェクトにおける支援金の目標額（currency で指定された通貨単位で換算）
    - launched: プロジェクトの開始日時
    - pledged: プロジェクトで実際に集められた支援額
    - state: プロジェクトのステータス（成功・失敗など）
    - backers: プロジェクトの支援者数
    - country: プロジェクトの主体の所属国
    - usd pledged: pledged をある時点でのレートで USD 換算したもの（明らかに誤っている値が多い）
    - usd_pledged_real: pledged をある時点でのレートで USD 換算したもの（usd pledged とは異なり，明らかに誤っている値は見られない）
    - usd_goal_real: goal をある時点でのレートで USD 換算したもの

In [3]:
df.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


### （参考）ks-projects-201612.csv の読み込み
- `encoding="ISO-8859-1"` を指定する必要がある

In [4]:
df2 = pd.read_csv('../data/ks-projects-201612.csv', encoding="ISO-8859-1")

C:\Users\ssshu\AppData\Local\Temp\ipykernel_10576\4057632774.py:1: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('../data/ks-projects-201612.csv', encoding="ISO-8859-1")


In [5]:
df2.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


## データの基礎集計

### データフレームの概要
- pandas.DataFrame の info メソッドを利用する

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378661 entries, 1000002330 to 999988282
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              378657 non-null  object 
 1   category          378661 non-null  object 
 2   main_category     378661 non-null  object 
 3   currency          378661 non-null  object 
 4   deadline          378661 non-null  object 
 5   goal              378661 non-null  float64
 6   launched          378661 non-null  object 
 7   pledged           378661 non-null  float64
 8   state             378661 non-null  object 
 9   backers           378661 non-null  int64  
 10  country           378661 non-null  object 
 11  usd pledged       374864 non-null  float64
 12  usd_pledged_real  378661 non-null  float64
 13  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 43.3+ MB


### 連続変数の基礎集計
- pandas.DataFrame の describe メソッドを利用する

In [7]:
df.describe()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


### （参考）離散変数の基礎集計
- pandas.Series の value_counts メソッドを利用する
    - ただし，カテゴリ数が大きいと省略されてしまうため，非推奨

In [8]:
# for col, dtype in df.dtypes.items():
#     if dtype == 'object':
#         print(col)
#         display(df[col].value_counts())

## 有効な説明変数の選択
以下は，説明変数として利用するのには不適切である．
- ID: 基本的に説明変数として利用しない．例えば，番号の若いデータには正例，そうでないデータには負例が集められているなど，恣意的に決定された値であることがある．その場合，リーケージに直結するため，説明変数として利用しないことが適切である．ID は，データフレームの作成時にインデックスナンバーとして読み込んでしまえば，誤って説明変数として利用することがないため安全である．
- pledged: プロジェクト終了時に判明する説明変数であり，リーケージを引き起こすため，利用しない．
- backers: プロジェクト終了時に判明する説明変数であり，リーケージを引き起こすため，利用しない．
- usd pledged: プロジェクト終了時に判明する説明変数であり，リーケージを引き起こすため，利用しない．
- usd_pledged_real: 目的変数である．削除はしない．
- goal: usd_goal_real と重複している．こちらは，通貨単位がバラバラなので利用しづらいと判断して削除．
- state: プロジェクト終了時に判明する説明変数であり，リーケージを引き起こすため，利用しない．

In [9]:
df = df.drop(columns=['pledged', 'backers', 'usd pledged', 'state', 'goal'])

In [10]:
df.head()

,name,category,main_category,currency,deadline,launched,country,usd_pledged_real,usd_goal_real
ID,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,GB,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,US,2421.0,30000.00
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,US,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,US,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,US,1283.0,19500.00


また，launched と deadline から，期間を表す変数 period を作成する

In [11]:
# 日時に関する変数を TimeStamp に変換
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])

In [12]:
# 期間の変数を作成
df['period'] = (df['deadline'] - df['launched']).dt.days 

## データの可視化及び欠損値・異常値の処理

### 連続変数の可視化及び異常値の処理
- seaborn の pairplot によって散布図行列を描画する
    - 描画には数分ほど時間がかかる場合がある

In [ ]:
sns.pairplot(df)

散布図行列から，以下のことがわかる．
- period が 10000 以上のデータが数件 -> 異常値として除去

In [ ]:
df = df[df['period'] < 10000] # 異常値の除去

In [ ]:
sns.pairplot(df) # 再描画

- ヒストグラムから，usd_goal_real および usd_pledged_real は右に大きく歪んだ分布を描くことがわかる -> 対数変換を検討

In [ ]:
epsilon = 1e-5 # 対数変換の際に負の無限大に発散しないようにフロアリングするパラメータ
df['log_usd_goal'] = df['usd_goal_real'].apply(lambda x: np.log10(x + epsilon))
df['log_usd_pledged'] = df['usd_pledged_real'].apply(lambda x: np.log10(x + epsilon))

sns.pairplot(df[['log_usd_pledged', 'log_usd_goal', 'period']])

### 離散変数の可視化，変数選択，及び異常値の処理

離散変数同士の関連性を見る．下記では，連関係数を用いた議論を行うが，講座範囲外であるためクロス集計表の目視によって変数選択をしても十分である．

In [ ]:
# object 型の変数名を表示
for col, dtype in df.dtypes.items():
    if dtype == 'object':
        print(col)

以下について，クロス集計表を作成し，[連関係数](https://qiita.com/shngt/items/45da2d30acf9e84924b7#クラメールの連関係数)を算出
- category と main_category
- currency と country

In [ ]:
# 連関係数を求める関数
def cramer_coef(x):
    chi2 = st.chi2_contingency(x)[0]
    return np.sqrt(chi2 / x.sum() / (min(x.shape) - 1))

In [ ]:
# category vs main_category
ct_category = pd.crosstab(df['category'], df['main_category'])
cramer_coef(ct_category.to_numpy())

非常に大きな連関係数が得られたため，片方の変数を用いれば十分であると判断する．変数選択の基準としては以下が考えられる．
- より細かな category を採用し，カテゴリ数削減のためにまとめる
- main_category を採用する

今回は簡単のため，後者とする

In [ ]:
df = df.drop(columns=['category'])

In [ ]:
# country vs currency
ct_country = pd.crosstab(df['country'], df['currency'])
cramer_coef(ct_country.to_numpy())

非常に大きな連関係数が得られたため，片方の変数を用いれば十分であると判断する．変数選択の基準としては以下が考えられる．
- country を採用する
- currency を採用する

この状況では判断が難しいので，後で判断することにする

### 欠損値の処理

In [ ]:
df.isnull().sum()

name 列から後ほど特徴量を作成することを考慮して，行を削除

In [ ]:
df = df.dropna(axis=0)

In [ ]:
# 再表示
df.isnull().sum()

In [ ]:
for col, dtype in df.dtypes.items():
    if dtype == 'object':
        print(col)
        display(df[col].value_counts())

country 列に`N,0"`なる不明な値があるので，分析する

In [ ]:
df[df['country'] == 'N,0"']['currency'].value_counts()

country は currency と強い連関があったため，country 列を削除しても良いと判断する

In [ ]:
df = df.drop(columns='country')

## 質的変数の処理
現状，質的変数として残存しているのは以下である
- name: 単語数を抽出
- main_category: ワンホットベクトルに変換
- currency: ワンホットベクトルに変換

In [ ]:
df['n_words'] = df['name'].apply(lambda x: len(str(x).split(' ')))

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['name', 'deadline', 'launched'])

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.info()

In [ ]:
# 作成したデータフレームを保存
df.to_csv('../data/df_regression.csv')

## 線形回帰

まずは対数変換前で線形回帰を行う．

In [ ]:
X = df.drop(columns=['log_usd_goal', 'log_usd_pledged', 'usd_pledged_real'])
y = df['usd_pledged_real']
lr_reg = LinearRegression()
lr_reg.fit(X, y)

## 識別精度の算出
- 検証精度の算出については，DAY2以降で取り扱う

In [ ]:
y_predicted = lr_reg.predict(X)

In [ ]:
mae = mean_absolute_error(y, y_predicted)
mse = mean_squared_error(y, y_predicted)
rmse = np.sqrt(mse)

print(f'MAE: {mae:.3}')
print(f'MSE: {mse:.3}')
print(f'RMSE: {rmse:.3}')

## 対数変換を利用した線形回帰
- 線形回帰は目的変数の誤差が正規分布に従うことを仮定したモデルであるため，各変数を対数変換した場合の挙動を調べる

In [ ]:
log_X = df.drop(columns=['usd_goal_real', 'log_usd_pledged', 'usd_pledged_real'])
log_y = df['log_usd_pledged']
log_lr_reg = LinearRegression()
log_lr_reg.fit(log_X, log_y)

In [ ]:
log_y_predicted = log_lr_reg.predict(log_X)

In [ ]:
# 対数変換後での誤差
mae = mean_absolute_error(log_y, log_y_predicted)
mse = mean_squared_error(log_y, log_y_predicted)
rmse = np.sqrt(mse)

print(f'MAE: {mae:.3}')
print(f'MSE: {mse:.3}')
print(f'RMSE: {rmse:.3}')

In [ ]:
# 対数変換前での誤差
mae = mean_absolute_error(10**log_y - epsilon, 10**log_y_predicted - epsilon)
mse = mean_squared_error(10**log_y - epsilon, 10**log_y_predicted - epsilon)
rmse = np.sqrt(mse)

print(f'MAE: {mae:.3}')
print(f'MSE: {mse:.3}')
print(f'RMSE: {rmse:.3}')

対数変換なしの場合と比べて，MAEは小さくなったが，MSE・RMSEは大きくなった